Let's do a quick example comparing `jitr` to the standard Runge-Kutta ODE solver implemented in `scipy.integrate.solve_ivp`:

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import njit
import jitr

First let's check our library versions and configs, as this will affect performance:

In [2]:
import scipy as sc

sc.__version__

'1.11.3'

In [3]:
import numba

numba.__version__

'0.57.1'

In [4]:
np.__version__

'1.24.4'

In [5]:
np.show_config()

blas_info:
    libraries = ['cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/home/kyle/mambaforge/envs/om/lib']
    include_dirs = ['/home/kyle/mambaforge/envs/om/include']
    language = c
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    define_macros = [('NO_ATLAS_INFO', 1), ('HAVE_CBLAS', None)]
    libraries = ['cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/home/kyle/mambaforge/envs/om/lib']
    include_dirs = ['/home/kyle/mambaforge/envs/om/include']
    language = c
lapack_info:
    libraries = ['lapack', 'blas', 'lapack', 'blas']
    library_dirs = ['/home/kyle/mambaforge/envs/om/lib']
    language = f77
lapack_opt_info:
    libraries = ['lapack', 'blas', 'lapack', 'blas', 'cblas', 'blas', 'cblas', 'blas']
    library_dirs = ['/home/kyle/mambaforge/envs/om/lib']
    language = c
    define_macros = [('NO_ATLAS_INFO', 1), ('HAVE_CBLAS', None)]
    include_dirs = ['/home/kyle/mambaforge/envs/om/include']
Supported SIMD extensions in this NumPy install:

Great, now let's set up our system and solver with `jitr`:

In [6]:
sys = jitr.ProjectileTargetSystem(
    np.array([939.0]),
    np.array([5 * (2 * np.pi)]),
    l=np.array([0]),
    Ztarget=40,
    Zproj=0,
    nchannels=1,
)

# COM frame energy
Ecom = 14.1

# Lagrange-Mesh R-matrix solver
solver = jitr.LagrangeRMatrixSolver(100, 1, sys, ecom=Ecom)

# ch holds info for the elastic scattering channel
ch = np.array(sys.build_channels(Ecom))

# use same interaction for all channels
interaction_matrix = jitr.InteractionMatrix(1)
interaction_matrix.set_local_interaction(jitr.woods_saxon_potential)

# Woods-Saxon potential parameters
V0 = 60  # real potential strength
W0 = 20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness
params = (V0, W0, R0, a0)

# set the params to be used when we run the solver
interaction_matrix.local_args[0, 0] = params

In [7]:
# run solver
R, S, uext_prime_boundary = solver.solve(interaction_matrix, ch, ecom=Ecom)

In [8]:
print(S[0][0])

(-0.056995279766827306-0.029788520925563822j)


Great, now let's use `scipy` and see if we get the same $\mathcal{S}$-matrix:

In [9]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1

sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    ch[0].domain,
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol

In [10]:
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.smatrix(R_rk, a, ch[0].l, ch[0].eta)
print(S_rk)

(-0.05699524506911337-0.02978858462683773j)


In [11]:
100 * (S[0, 0] - S_rk) / S_rk  # percent difference in real and imag parts of S

(1.9351413848065906e-06-0.00011277734670611897j)

Great, our solvers agree up to high precision. Now let's compare the runtime of the two solver options:

In [12]:
%%timeit
R, S, uext_prime_boundary = solver.solve(interaction_matrix, ch, ecom=Ecom)

2.92 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, ch[0], interaction_matrix.local_matrix[0, 0], params
    ),
    (ch[0].domain[0], ch[0].domain[1] * 1.2),
    ch[0].initial_conditions(),
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol
a = ch[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = jitr.smatrix(R_rk, a, ch[0].l, ch[0].eta)

91.7 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


On my machine `jitr` is faster by about 50 times!

(This does, of course, depend on the solver paramaters; `atol` and `rtol` for `solve_ivp`, and `nbasis` for `LagrangeRMatrixSolver` ).